In [1]:
import import_ipynb
import g1_generate_dictionary as dict_gen
import e2_extract_occ_original as stat_original
import e1_extract_occ_gen as stat_gen
import numpy as np
import sys
import csv
import numpy.ma as ma

importing Jupyter notebook from g1_generate_dictionary.ipynb
importing Jupyter notebook from e2_extract_occ_original.ipynb
importing Jupyter notebook from e1_extract_occ_gen.ipynb


In [2]:
def extract_stat_orig(dictionary,orig_occ):
     """Take the occurences for only the entities in the dicitonary for the original dataset"""
    stat_original = []
    for o in orig_occ:
        if(len(dictionary[o[0]]) !=0 ):
            stat_original.append((o[0],o[1]))
    return stat_original

In [3]:
def extract_stat_gen(dictionary, gen_occ):
    """Take the occurences for only the entities in the dicitonary for the generated dataset"""
    stat_g = []
    for o in gen_occ:
        if(o[0] in sum(dictionary.values(),[])):
            stat_g.append((o[0],o[1]))
    return stat_g

In [4]:
def combine_gen(dictionary, original_occ, generated_occ):
    """Combine the entities that have different spellings in generated_occ with the dictionary"""
    combined = []
    for eo in original_occ:
        list_eo = []
        for eg in generated_occ:
            if(eg[0] in dictionary[eo[0]]):
                list_eo.append(eg)
        combined.append(list_eo)
    return combined

In [6]:
def compare_ori_gen(generated_occ, original_occ):
    """From all the entities in generated_occ keep only the one with the max number of interventions. Return both generated and original occurences with the same order inside (index i : both the same entity's interventions)"""
    original = []
    generated = []
    for i in range(len(original_occ)):
        original.append(original_occ[i][1])
        generated.append(sorted(generated_occ[i],key=lambda x: x[1], reverse=True)[0][1])

    return original,generated 

In [7]:
def extract_interventions(number, list_sentences):
    """Extract the intervention for both dataset and keep only the one that match the dictionary for one issue"""
    #Load the occurences for the original and generated file, and the dictionary
    original_occ = stat_original.extract_occurences_issue(number)
    gen_occ = stat_gen.extract_occurences_issue_ENB(list_sentences)
    dictionary = dict_gen.compute_dictionary()
    
    #Take only entities in the dictionary (to have same length)
    original_occ = extract_stat_orig(dictionary,original_occ)
    generated_occ = extract_stat_gen(dictionary, gen_occ)

    #Combine element same key
    generated_occ= combine_gen(dictionary, original_occ, generated_occ)

    # x: intervention of the original file
    # y: intervention of the generated file 
    x,y = compare_ori_gen(generated_occ, original_occ)

    return x,y 

In [8]:
def compute_correlation_coefficient(number, list_sentences):
    """Compute the coeeficient of correlation of intervention between the dataset for one speicific issue"""
    x,y = extract_interventions(number, list_sentences)
    x =  np.array(x).astype(np.float)
    y = np.array(y).astype(np.float)

    return np.corrcoef(x,y)